In [1]:
from models import common

domain = env['mail_reset.domain'].search([])
sql = f'SELECT name,email_other,local_part,domain,quota,active from mailbox WHERE domain = "{domain.name.lower()}";'
output = common._run_sql_on_maildb(domain.api_url, domain.api_token, domain.namespace, domain.label, sql)
fields = ['name','recovery_email','username','domain','quota','active']
records = common._get_record_data(output, fields)
# Cleanup all active/inactive alias records
all_mailboxes = env['mail_reset.users'].with_context(active_test=False).search([])
for mailbox in all_mailboxes:
    mailbox.unlink()

for data in records:
    data['quota'] = common._reverse_quota_value(data['quota'])
    data['domain'] = env['mail_reset.domain'].search([('name','=', data['domain'])]).id

    mailbox = env['mail_reset.users'].create(data)

    if mailbox:
        print(f"{mailbox.id}: {mailbox.name}@{mailbox.domain.name} created")


mysql -u postfix -p$MYSQL_PASSWORD -D postfix -e 'SELECT name,email_other,local_part,domain,quota,active from mailbox WHERE domain = "exrp.pl";'
mysql -u postfix -p$MYSQL_PASSWORD -D postfix -e 'INSERT mailbox (name,username,email_other,password,maildir,local_part,domain,quota) VALUES("Andriy Sukhoversha","a_suh@exrp.pl","asukhoversha@gmail.com","xWDt^R$b","exrp.pl/a_suh/","a_suh","exrp.pl","0");INSERT alias (address,goto,domain) VALUES("a_suh@exrp.pl","a_suh@exrp.pl","exrp.pl");'
2: Andriy Sukhoversha@exrp.pl created
mysql -u postfix -p$MYSQL_PASSWORD -D postfix -e 'INSERT mailbox (name,username,email_other,password,maildir,local_part,domain,quota) VALUES("Catchall","catchall@exrp.pl","nurlan@erpgo.az","6W?NjbuJ","exrp.pl/catchall/","catchall","exrp.pl","307200000");INSERT alias (address,goto,domain) VALUES("catchall@exrp.pl","catchall@exrp.pl","exrp.pl");'
3: Catchall@exrp.pl created
mysql -u postfix -p$MYSQL_PASSWORD -D postfix -e 'INSERT mailbox (name,username,email_other,password,

In [10]:
output

'name\temail_other\tlocal_part\tdomain\tquota\tactive\nAndriy Sukhoversha\tasukhoversha@gmail.com\ta_suh\texrp.pl\t0\t1\nCatchall\tnurlan@erpgo.az\tcatchall\texrp.pl\t307200000\t1\nGreat user\tsuhnewmail@gmail.com\tg_user\texrp.pl\t51200000\t1\nnormal user\ttest@example.com\tnormal\texrp.pl\t10240000\t0\nNurlan Farajov\tcoolinuxoid@gmail.com\tnurlan.farajov\texrp.pl\t204800000\t1\nNurlan test\tcoolinuxoid@gmail.com\tnurlantest\texrp.pl\t10240000\t1\n'

In [ ]:
def sync_aliases(domain):
    sql = f'SELECT name,username,email_other,local_part,domain,quota,active WHERE domain = "{domain.name.lower()}";'
    output = common._run_sql_on_maildb(domain.api_url, 
                                       domain.api_token, 
                                       domain.namespace, 
                                       domain.label, sql)

    fields = ['name','goto','domain','active']
    records = common._get_record_data(output, fields)

    # Cleanup all active/inactive alias records
    all_aliases = env['mail_reset.aliases'].with_context(active_test=False).search([])
    for alias in all_aliases:
        alias.unlink()

    for data in records:
        if data['name'] == '':
            data['name'] = '*'
        data['goto'] = common._comma_to_newline(data['goto'])
        data['domain'] = env['mail_reset.domain'].search([('name','=', data['domain'])]).id

        alias = env['mail_reset.aliases'].create(data)

        if alias:
            print(f"{alias.id}: {alias.name}@{alias.domain.name} created")
